---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
import pandas as pd
import numpy as np

def blight_model():
    import pandas as pd
    import numpy as np
    from sklearn.preprocessing import MinMaxScaler

    train=pd.read_csv("train.csv", encoding = 'ISO-8859-1')
    test=pd.read_csv("test.csv")

    # Load address & latlons from .csv and join
    latlons = pd.read_csv('latlons.csv')
    address= pd.read_csv('addresses.csv')
    address=address.set_index('address').join(latlons.set_index('address'),how='left')
    address.head()

    #Remove redundant column in training set
    redundant_list=['payment_amount','payment_date','payment_status','balance_due','collection_status','compliance_detail']
    train.drop(redundant_list,axis=1,inplace=True)

    # Remove string columns
    string_columns=['agency_name','inspector_name','violator_name','violation_street_name',
                    'mailing_address_str_name','city','state','zip_code','country','ticket_issued_date',
                    'hearing_date','violation_code','violation_description','disposition','violation_zip_code',
                    'non_us_str_code','violation_street_number','mailing_address_str_number','grafitti_status']
    train.drop(string_columns,axis=1,inplace=True)
    test.drop(string_columns,axis=1,inplace=True)

    # Join train/test dataframes with ticket_id
    train=train.set_index('ticket_id').join(address.set_index('ticket_id'),how='left')
    test=test.set_index('ticket_id').join(address.set_index('ticket_id'),how='left')

    # Get rid of records with 'NaN' compliance
    train=train[(train.compliance==0)|(train.compliance==1)]

    # Get rid of records with 'Nan' lat/lon
    train.lat.fillna(method='pad',inplace=True)
    test.lat.fillna(method='pad',inplace=True)
    train.lon.fillna(method='pad',inplace=True)
    test.lon.fillna(method='pad',inplace=True)

    # Get rid of records with 'Nan' fine_amount
    train.fine_amount.fillna(method='pad',inplace=True)
    # Train/Test splits
    y_train=train.compliance
    X_train=train.drop('compliance',axis=1)
    X_test=test.copy()
    # Build Model
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import roc_auc_score
    clf = RandomForestRegressor(random_state=0)
    clf.fit(X_train,y_train)
    prediction=clf.predict(X_test)
    # GridSearch
    from sklearn.model_selection import GridSearchCV
    grid_values = {'max_features':[8,9],'max_depth': [8,9,10]}
    grid_clf_auc = GridSearchCV(clf, param_grid = grid_values, scoring = 'roc_auc')
    grid_clf_auc.fit(X_train, y_train)
    prediction=grid_clf_auc.predict(test)
    result = pd.Series(prediction, index=test.index)
    # Your code here
    
    return result# Your answer here

In [2]:
blight_model()

C:\Users\17520\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2907: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
C:\Users\17520\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


ticket_id
284932    0.049037
285362    0.026384
285361    0.061781
285338    0.055320
285346    0.055320
285345    0.055320
285347    0.059274
285342    0.394614
285530    0.028373
284989    0.028048
285344    0.062900
285343    0.024956
285340    0.021911
285341    0.094417
285349    0.073172
285348    0.073172
284991    0.028048
285532    0.041770
285406    0.027770
285001    0.047246
285006    0.047246
285405    0.020170
285337    0.026883
285496    0.054460
285497    0.060788
285378    0.027771
285589    0.041028
285585    0.056287
285501    0.064312
285581    0.027771
            ...   
376367    0.024527
376366    0.029962
376362    0.030074
376363    0.062749
376365    0.024527
376364    0.029962
376228    0.086624
376265    0.029962
376286    0.316308
376320    0.034216
376314    0.018925
376327    0.377337
376385    0.332571
376435    0.590658
376370    0.384340
376434    0.052473
376459    0.069255
376478    0.007815
376473    0.036387
376484    0.039266
376482    0.030647
37

In [3]:
# Playground
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

train=pd.read_csv("train.csv", encoding = 'ISO-8859-1')
test=pd.read_csv("test.csv")
train.head()

C:\Users\17520\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
3,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
4,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN


In [4]:
# Load address & latlons from .csv and join
latlons = pd.read_csv('latlons.csv')
address= pd.read_csv('addresses.csv')
address=address.set_index('address').join(latlons.set_index('address'),how='left')
address.head()

,ticket_id,lat,lon
address,,,
"-11064 gratiot, Detroit MI",328722,42.406935,-82.995599
"-11871 wilfred, Detroit MI",350971,42.411288,-82.993674
"-15126 harper, Detroit MI",344821,42.406402,-82.957525
"0 10th st, Detroit MI",24928,42.325689,-83.064330
"0 10th st, Detroit MI",71887,42.325689,-83.064330


In [5]:
#Remove redundant column in training set
redundant_list=['payment_amount','payment_date','payment_status','balance_due','collection_status','compliance_detail']
train.drop(redundant_list,axis=1,inplace=True)

In [6]:
train.columns

Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'grafitti_status', 'compliance'],
      dtype='object')

In [7]:
# Print all column data types
for column in list(train.columns):
    print(type(train[str(column)][1]),column)

<class 'numpy.int64'> ticket_id
<class 'str'> agency_name
<class 'str'> inspector_name
<class 'str'> violator_name
<class 'numpy.float64'> violation_street_number
<class 'str'> violation_street_name
<class 'numpy.float64'> violation_zip_code
<class 'numpy.float64'> mailing_address_str_number
<class 'str'> mailing_address_str_name
<class 'str'> city
<class 'str'> state
<class 'int'> zip_code
<class 'float'> non_us_str_code
<class 'str'> country
<class 'str'> ticket_issued_date
<class 'str'> hearing_date
<class 'str'> violation_code
<class 'str'> violation_description
<class 'str'> disposition
<class 'numpy.float64'> fine_amount
<class 'numpy.float64'> admin_fee
<class 'numpy.float64'> state_fee
<class 'numpy.float64'> late_fee
<class 'numpy.float64'> discount_amount
<class 'numpy.float64'> clean_up_cost
<class 'numpy.float64'> judgment_amount
<class 'float'> grafitti_status
<class 'numpy.float64'> compliance


In [8]:
# Remove string columns
string_columns=['agency_name','inspector_name','violator_name','violation_street_name',
                'mailing_address_str_name','city','state','zip_code','country','ticket_issued_date',
                'hearing_date','violation_code','violation_description','disposition','violation_zip_code',
                'non_us_str_code','violation_street_number','mailing_address_str_number','grafitti_status']
train.drop(string_columns,axis=1,inplace=True)
test.drop(string_columns,axis=1,inplace=True)

In [9]:
# # Join train/test dataframes with ticket_id
train=train.set_index('ticket_id').join(address.set_index('ticket_id'),how='left')
test=test.set_index('ticket_id').join(address.set_index('ticket_id'),how='left')

In [10]:
# print('Columns contain NaN value in train set')
for column_name in train.columns:
    if(train[column_name].isnull().any()):
        print(column_name)
print('Columns contain NaN value in test set')
for column_name in test.columns:
    if(test[column_name].isnull().any()):
        print(column_name)

fine_amount
compliance
lat
lon
Columns contain NaN value in test set
lat
lon


In [11]:
print('The number of null values of lat/lon in train set',
train['lat'].isnull().value_counts()[1],';',train['lon'].isnull().value_counts()[1])
print('The number of null values of lat/lon in test set',
test['lat'].isnull().value_counts()[1],';',test['lon'].isnull().value_counts()[1])

The number of null values of lat/lon in train set 3 ; 3
The number of null values of lat/lon in test set 5 ; 5


In [12]:
# Get rid of records with 'NaN' compliance
train=train[(train.compliance==0)|(train.compliance==1)]
train.compliance.isnull().any()

False

In [13]:
# Get rid of records with 'Nan' lat/lon
train.lat.fillna(method='pad',inplace=True)
test.lat.fillna(method='pad',inplace=True)
train.lon.fillna(method='pad',inplace=True)
test.lon.fillna(method='pad',inplace=True)

In [14]:
#Get rid of records with 'Nan' fine_amount
train.fine_amount.fillna(method='pad',inplace=True)

In [15]:
# Create train/test split
from sklearn.model_selection import train_test_split

# X_train,X_cross,y_train,y_cross=train_test_split(train.drop('compliance',axis=1),train['compliance'])

y_train=train['compliance']
X_train=train.drop('compliance',axis=1)

In [16]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(test)

In [17]:
# Choosing model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_auc_score
clf = RandomForestRegressor(random_state=0)
# clf.fit(X_train,y_train)
# prediction_cross=clf.predict(X_cross)
# prediction_train=clf.predict(X_train)
# print(roc_auc_score(y_cross,prediction_cross))
# print(roc_auc_score(y_train,prediction_train))

In [18]:
from sklearn.model_selection import GridSearchCV
grid_values = {'max_features':[8,9],'max_depth': [8,9,10]}
grid_clf_auc = GridSearchCV(clf, param_grid = grid_values,n_jobs=4, scoring = 'roc_auc')
grid_clf_auc.fit(X_train, y_train)
print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
print('Grid best score (AUC): ', grid_clf_auc.best_score_)

Grid best parameter (max. AUC):  {'max_depth': 9, 'max_features': 9}
Grid best score (AUC):  0.7628255763249424
